<a href="https://colab.research.google.com/github/nesliyl/datasettrain/blob/main/preparemodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import missingno as msno
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
import json
import re
import torch

from transformers import BertTokenizer, BertModel

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from difflib import get_close_matches



import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
!pip install faiss-cpu
import faiss

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from ast import literal_eval

In [ ]:
from urllib.parse import urlparse

In [ ]:
!pip install pycountry
import pycountry

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 56.3 MB/s eta 0:00:00


In [ ]:
import shutil

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from imblearn.over_sampling import SMOTE
from collections import Counter


In [ ]:
from imblearn.over_sampling import SMOTENC

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
from xgboost import XGBClassifier

In [ ]:
from sklearn.metrics import roc_auc_score


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("CUDA kullanılıyor. GPU ile işlem yapılacak.")
    batch_size = 512
else:
    device = torch.device("cpu")
    print("CUDA desteklenmiyor, işlemler CPU'da gerçekleştirilecek.")

CUDA kullanılıyor. GPU ile işlem yapılacak.


In [ ]:
import os
os.environ["HF_HOME"] = "/root/.cache/huggingface"


nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df=pd.read_csv("startupdataset.csv")

In [ ]:
df.isnull().sum()

,0
permalink,0
name,1
homepage_url,5058
category_list,3148
funding_total_usd,0
status,0
country_code,6958
state_code,8547
region,8030
city,8028


In [ ]:
json_file = "crunchbase_filtered_results.json"
with open(json_file, "r", encoding="utf-8") as f:
    json_data = json.load(f)

# JSON verisini permalink bazında dictionary formatına çevir
json_dict = {f"/organization/{item['permalink']}": item["short_description"] for item in json_data}

# Category_list sütunu boş olan ve permalink eşleşen satırları güncelle
for index, row in df.iterrows():
    if pd.isna(row["category_list"]) and row["permalink"] in json_dict:
        df.at[index, "category_list"] = json_dict[row["permalink"]]

# Güncellenmiş CSV dosyasını kaydet
df.to_csv("updated_startupdataset.csv", index=False)


In [ ]:
df=pd.read_csv("updated_startupdataset.csv")


df = df.dropna(subset=['category_list'])

df.to_csv("updated_startupdataset.csv", index=False)


In [ ]:
df.isnull().sum()

,0
permalink,0
name,1
homepage_url,4852
category_list,0
funding_total_usd,0
status,0
country_code,6649
state_code,8231
region,7714
city,7712


In [ ]:
def clean_text(text):
    text = str(text).lower()
    text = text.replace('|', ' ')
    text = re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', text)
    text = re.sub(r'[^a-z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

df['category_list'] = df['category_list'].apply(clean_text)


def tokenize_and_remove_stopwords(text):
    words = text.split()
    words = [word for word in words if word not in stop_words]
    words = [stemmer.stem(word) for word in words]
    return words

df['category_tokens'] = df['category_list'].apply(tokenize_and_remove_stopwords)
df.to_csv("preprocessed_startupdataset.csv", index=False)

In [ ]:
df=pd.read_csv("preprocessed_startupdataset.csv")

# bert
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", cache_dir="/root/.cache/huggingface")
model = BertModel.from_pretrained("bert-base-uncased", cache_dir="/root/.cache/huggingface").to(device)
batch_size = 32
def batch_bert_embedding(text_list):
    inputs = tokenizer(text_list, return_tensors="pt", padding=True, truncation=True, max_length=10)
    inputs = {key: val.to(device) for key, val in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state[:, 0, :].cpu().tolist()


embeddings = []
for i in range(0, len(df), batch_size):
    batch_texts = df['category_tokens'][i:i+batch_size].tolist()
    batch_embeddings = batch_bert_embedding(batch_texts)
    embeddings.extend(batch_embeddings)

df['category_embedding'] = embeddings

# ilk embedding degeri
print("İlk embedding değeri:", df['category_embedding'].iloc[0])


df.to_csv("preprocessed_startupdataset.csv", index=False)

print("Ön işleme tamamlandı: Stopwords temizlendi, kelimeler köklerine indirildi, BERT vektörleri batch işleme ile çıkarıldı. Dosya kaydedildi: preprocessed_startupdataset.csv")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

İlk embedding değeri: [0.12721127271652222, 0.17106027901172638, -0.08594249933958054, -0.23734673857688904, -0.27755317091941833, -0.33221322298049927, 0.22691935300827026, 0.2778116464614868, -0.3725869357585907, -0.09925819933414459, -0.36772143840789795, 0.18280747532844543, -0.02741527184844017, 0.29849621653556824, 0.5323564410209656, 0.24069736897945404, -0.2989821135997772, 0.2881543040275574, 0.3228912353515625, -0.36016228795051575, -0.11076338589191437, -0.06548774987459183, -0.10250464081764221, -0.23778215050697327, -0.03863394632935524, -0.06835698336362839, 0.2556473910808563, -0.34545719623565674, -0.206440269947052, 0.44771939516067505, -0.06104670464992523, 0.18020114302635193, -0.2473464459180832, 0.26183849573135376, 0.4510001540184021, -0.07755320519208908, 0.12564322352409363, 0.09406585991382599, 0.17569506168365479, 0.20377516746520996, 0.10287623852491379, 0.12085306644439697, 0.2138552963733673, -0.17451660335063934, -0.10471345484256744, -0.18020811676979065,

In [ ]:
def safe_convert_embedding(x):
    try:
        if isinstance(x, str):
            return np.array(literal_eval(x), dtype=np.float32)
        elif isinstance(x, list):
            return np.array(x, dtype=np.float32)
        else:
            return np.nan
    except Exception:
        return np.nan



def fill_missing_funding(df):
    df_filled = df.copy()


    df_filled['funding_total_usd'] = df_filled['funding_total_usd'].replace('-', np.nan)
    df_filled['funding_total_usd'] = pd.to_numeric(df_filled['funding_total_usd'], errors='coerce')


    df_filled['category_embedding'] = df_filled['category_embedding'].astype(str).apply(safe_convert_embedding)


    df_missing = df_filled[df_filled['funding_total_usd'].isna()].copy()
    df_known = df_filled.dropna(subset=['funding_total_usd']).copy()


    df_known = df_known.dropna(subset=['category_embedding']).copy()
    known_embeddings = np.stack(df_known['category_embedding'].values)
    missing_embeddings = np.stack(df_missing['category_embedding'].dropna().values)

    # FAISS ile hızlı nearest neighbor hesaplaması
    d = known_embeddings.shape[1]
    index = faiss.IndexFlatL2(d)
    index.add(known_embeddings)


    _,nearest_indices = index.search(missing_embeddings,1)

    missing_funding_values = {}
    for idx, row in enumerate(df_missing.itertuples()):
        predicted_value = df_known.iloc[nearest_indices[idx][0]]['funding_total_usd']
        missing_funding_values[row.permalink] = predicted_value


    for permalink, predicted_value in missing_funding_values.items():
        df_filled.loc[df_filled['permalink'] == permalink, 'funding_total_usd'] = predicted_value


    df_filled.to_csv("preprocessed_startupdataset.csv", index=False)
    print("Tam veri seti preprocessed_startupdataset.csv olarak güncellendi.")

    return df_filled


df = fill_missing_funding(df)

print("İşlem tamamlandı. FAISS kullanarak en hızlı şekilde funding_total_usd değerleri tahmin edilerek güncellendi.")

Tam veri seti preprocessed_startupdataset.csv olarak güncellendi.
İşlem tamamlandı. FAISS kullanarak en hızlı şekilde funding_total_usd değerleri tahmin edilerek güncellendi.


In [ ]:
#eksik değerleri yalnızca updated_startupdataset.csv'ye kaydetme -> kopya

df_updated = pd.read_csv("updated_startupdataset.csv")


for idx, row in df_updated.iterrows():
    if row['funding_total_usd'] == '-':
        matching_row = df[df['permalink'] == row['permalink']]
        if not matching_row.empty:
            df_updated.at[idx, 'funding_total_usd'] = matching_row['funding_total_usd'].values[0]


df_updated.to_csv("updated_startupdataset.csv", index=False)
print("Eksik funding_total_usd değerleri updated_startupdataset.csv dosyasına aktarıldı.")

Eksik funding_total_usd değerleri updated_startupdataset.csv dosyasına aktarıldı.


In [ ]:
df=pd.read_csv("updated_startupdataset.csv")
df.isnull().sum()

,0
permalink,0
name,1
homepage_url,4852
category_list,0
funding_total_usd,0
status,0
country_code,6649
state_code,8231
region,7714
city,7712


In [ ]:
preprocessed_file = "preprocessed_startupdataset.csv"
updated_file = "updated_startupdataset.csv"

df_preprocessed = pd.read_csv(preprocessed_file)
df_updated = pd.read_csv(updated_file)


def extract_domain(url):
    try:
        if pd.isna(url) or not isinstance(url, str):
            return None, None

        domain = urlparse(url).netloc
        domain_parts = domain.split('.')


        if len(domain_parts) < 2:
            return domain, None

        # ülke kodu olup olmadığını kontrol eden blok
        if len(domain_parts) > 2 and len(domain_parts[-1]) == 2:  # .uk, .fr gibi kodlar 2 harfli bunları ülke kodu olarak kabul ediyoruz.
            country_code = domain_parts[-1]
        else:
            country_code = None

        return domain, country_code
    except Exception:
        return None, None


df_missing_country = df_updated[df_updated['country_code'].isna() & df_updated['homepage_url'].notna()].copy()


df_missing_country[['domain', 'estimated_country']] = df_missing_country['homepage_url'].apply(lambda x: pd.Series(extract_domain(x)))


for index, row in df_missing_country.iterrows():
    if pd.notna(row['estimated_country']):
        df_preprocessed.loc[df_preprocessed['permalink'] == row['permalink'], 'country_code'] = row['estimated_country']
        df_updated.loc[df_updated['permalink'] == row['permalink'], 'country_code'] = row['estimated_country']


df_preprocessed.to_csv("preprocessed_startupdataset.csv", index=False)
df_updated.to_csv("updated_startupdataset.csv", index=False)

print("eksik ülke kodları başarıyla tahmin edildi ve her iki dataset güncellendi!")

In [ ]:
df=pd.read_csv("updated_startupdataset.csv")
df.isnull().sum()

,0
permalink,0
name,1
homepage_url,4852
category_list,0
funding_total_usd,0
status,0
country_code,5859
state_code,8231
region,7714
city,7712


In [ ]:
preprocessed_file = "preprocessed_startupdataset.csv"
updated_file = "updated_startupdataset.csv"

df_preprocessed = pd.read_csv(preprocessed_file)
df_updated = pd.read_csv(updated_file)

# ISO 3166-1 alpha-3 veya yanlış formatta olanları alpha-2'ye çevirme fonksiyon
def convert_to_iso2(code, lowercase=False):
    try:
        if isinstance(code, str):
            code = code.upper()
            country = pycountry.countries.get(alpha_3=code)
            if country:
                return country.alpha_2.lower() if lowercase else country.alpha_2
        return code if len(code) == 2 else None
    except:
        return None

df_updated['country_code'] = df_updated['country_code'].apply(lambda x: convert_to_iso2(x, lowercase=False))


df_preprocessed['country_code'] = df_preprocessed['country_code'].apply(lambda x: convert_to_iso2(x, lowercase=True))


df_updated.to_csv("updated_startupdataset.csv", index=False)
df_preprocessed.to_csv("preprocessed_startupdataset.csv", index=False)

print("ulke kodları 2 harfli ISO-2 formatına çevrildi ve her iki dataset güncellendi!")

In [ ]:
preprocessed_file = "preprocessed_startupdataset.csv"
df_preprocessed = pd.read_csv(preprocessed_file)

# ulke kodu var city olmayanların sayısı
missing_city_count = df_preprocessed[(df_preprocessed['country_code'].notna()) & (df_preprocessed['city'].isna())].shape[0]

print(f"ulke kodu olup city bilgisi olmayan veri sayısı: {missing_city_count}")

In [ ]:
#file_path = "preprocessed_startupdataset.csv"
#df = pd.read_csv(file_path)


#most_common_cities = df.groupby('country_code')['city'].agg(lambda x: x.mode()[0] if not x.mode().empty else None)


#df['city'] = df.apply(
 #   lambda row: most_common_cities[row['country_code']] if pd.isna(row['city']) and pd.notna(row['country_code']) else row['city'],
  #  axis=1
#)


#missing_city_count_after = df[(df['country_code'].notna()) & (df['city'].isna())].shape[0]

#print(f"✅ Güncelleme tamamlandı! Eksik city bilgisi kalan veri sayısı: {missing_city_count_after}")


#df.to_csv("preprocessed_startupdataset.csv", index=False)
#print("🚀 Dosya başarıyla güncellendi!")
updated_path = "updated_startupdataset.csv"
preprocessed_path = "preprocessed_startupdataset.csv"

df_updated = pd.read_csv(updated_path)
df_preprocessed = pd.read_csv(preprocessed_path)

# **2️⃣ updated_startupdataset içindeki city bilgisi boş olanları bul**
missing_city_permalinks = df_updated[df_updated['city'].isna()]['permalink'].tolist()

# **3️⃣ Preprocessed dataset içinde bu permalinke sahip olan satırları bul ve city bilgisi varsa boşalt**
df_preprocessed.loc[df_preprocessed['permalink'].isin(missing_city_permalinks), 'city'] = None

missing_city_count = df_preprocessed[(df_preprocessed['country_code'].notna()) & (df_preprocessed['city'].isna())].shape[0]

print(f"ulke kodu olup city bilgisi olmayan veri sayısı: {missing_city_count}")
df_preprocessed.to_csv("preprocessed_startupdataset.csv", index=False)

In [ ]:
df= pd.read_csv("preprocessed_startupdataset.csv")
missing_city_count = df_preprocessed[(df_preprocessed['country_code'].notna()) & (df_preprocessed['city'].isna())].shape[0]
print(f"ulke kodu olup city bilgisi olmayan veri sayısı: {missing_city_count}")

In [ ]:
original_file = "preprocessed_startupdataset.csv"


backup_file = "copy_preprocessed_startupdataset.csv"

shutil.copy(original_file, backup_file)

'copy_preprocessed_startupdataset.csv'

In [ ]:
file_path = "preprocessed_startupdataset.csv"
df = pd.read_csv(file_path)


missing_city_count_before = df[(df['country_code'].notna()) & (df['city'].isna())].shape[0]
print(f" Güncelleme öncesi eksik city bilgisi sayısı: {missing_city_count_before}")


most_common_cities = df[df['city'].notna()].groupby('country_code')['city'].agg(lambda x: x.mode()[0] if not x.mode().empty else None).to_dict()


def fill_missing_city(row):
    if pd.isna(row['city']) and pd.notna(row['country_code']):  # eger şehir eksikse ve ülke kodu varsa
        return most_common_cities.get(row['country_code'], None)  # O ülkeye ait en sık şehir varsa ata
    return row['city']

df.loc[df['city'].isna() & df['country_code'].notna(), 'city'] = df[df['city'].isna() & df['country_code'].notna()].apply(fill_missing_city, axis=1)


missing_city_count_after = df[(df['country_code'].notna()) & (df['city'].isna())].shape[0]
filled_count = missing_city_count_before - missing_city_count_after

print(f"guncelleme tamamlandı! Eksik city bilgisi kalan veri sayısı: {missing_city_count_after}")
print(f"doldurulan şehir bilgisi sayısı: {filled_count}")


df.to_csv("preprocessed_startupdataset.csv", index=False)

In [ ]:
df=pd.read_csv("preprocessed_startupdataset.csv")
df.isnull().sum()

,0
permalink,0
name,1
homepage_url,4852
category_list,0
funding_total_usd,0
status,0
country_code,5905
state_code,8231
region,7714
city,6658


In [ ]:
file_path = "preprocessed_startupdataset.csv"
df = pd.read_csv(file_path)


df_cleaned = df.dropna(subset=['first_funding_at'])

df_cleaned.to_csv("preprocessed_startupdataset.csv", index=False)

In [ ]:
df=pd.read_csv("preprocessed_startupdataset.csv")
df.isnull().sum()

,0
permalink,0
name,1
homepage_url,4852
category_list,0
funding_total_usd,0
status,0
country_code,5903
state_code,8229
region,7712
city,6656


In [ ]:
file_path = "preprocessed_startupdataset.csv"
df = pd.read_csv(file_path)


df['country_code'].fillna('unknown', inplace=True)


df.to_csv("preprocessed_startupdataset.csv", index=False)


unknown_count = (df['country_code'] == 'unknown').sum()
print(f"güncelleme tamamlandi,  'unknown' olarak atanan ülke kodu sayısı: {unknown_count}")
print("dosya başarıyla güncellendi ve kaydedildi!")

In [ ]:
df=pd.read_csv("preprocessed_startupdataset.csv")
df.isnull().sum()

,0
permalink,0
name,1
homepage_url,4852
category_list,0
funding_total_usd,0
status,0
country_code,0
state_code,8229
region,7712
city,6656


In [ ]:
original_file = "preprocessed_startupdataset.csv"


backup_file = "copy2_preprocessed_startupdataset.csv"


shutil.copy(original_file, backup_file)

'copy2_preprocessed_startupdataset.csv'

In [ ]:
file_path = "preprocessed_startupdataset.csv"
df = pd.read_csv(file_path)
df['city'].fillna('unknown', inplace=True)


df.to_csv("preprocessed_startupdataset.csv", index=False)

print("eksik şehir bilgileri 'unknown' olarak dolduruldu ve dosya güncellendi!")

In [ ]:
df=pd.read_csv("preprocessed_startupdataset.csv")
df.isnull().sum()

,0
permalink,0
name,1
homepage_url,4852
category_list,0
funding_total_usd,0
status,0
country_code,0
state_code,8229
region,7712
city,0


In [ ]:
file_path = "preprocessed_startupdataset.csv"
df = pd.read_csv(file_path)


df['first_funding_at'] = pd.to_datetime(df['first_funding_at'], format='%Y-%m-%d', errors='coerce')
df['last_funding_at'] = pd.to_datetime(df['last_funding_at'], format='%Y-%m-%d', errors='coerce')


df['years_since_first_funding'] = (pd.Timestamp.today() - df['first_funding_at']).dt.days / 365
df['years_since_last_funding'] = (pd.Timestamp.today() - df['last_funding_at']).dt.days / 365


print(df[['first_funding_at', 'last_funding_at', 'years_since_first_funding', 'years_since_last_funding']].head())

df.to_csv("preprocessed_startupdataset.csv", index=False)
print("dosya güncellendi,  tarih bazlı yıllar eklendi.")

In [ ]:
file_path = "preprocessed_startupdataset.csv"
df = pd.read_csv(file_path)

# funding_rounds sütunundaki minimum ve maksimum değerleri bul
min_funding_rounds = df['funding_rounds'].min()
max_funding_rounds = df['funding_rounds'].max()


print(min_funding_rounds, max_funding_rounds)

1 19


In [ ]:
file_path = "preprocessed_startupdataset.csv"
df = pd.read_csv(file_path)


min_funding_rounds = df['funding_total_usd'].min()
max_funding_rounds = df['funding_total_usd'].max()


print(min_funding_rounds, max_funding_rounds)

1.0 30079503000.0


In [ ]:
preprocessed_file = "preprocessed_startupdataset.csv"
model_file = "formodel_dataset.csv"

#kullanılacak sutun bilgileri
selected_columns = [
    "funding_total_usd",
    "status",
    "country_code",
    "city",
    "funding_rounds",
    "first_funding_at",
    "last_funding_at",
    "category_embedding"
]


df_preprocessed = pd.read_csv(preprocessed_file, usecols=selected_columns)


df_preprocessed.to_csv(model_file, index=False)



In [ ]:
df_model = pd.read_csv("formodel_dataset.csv")


print("model için seçilen ilk 20 veri:")
print(df_model.head(20))
df_model = pd.read_csv("formodel_dataset.csv")


print(f"model veri setinde toplam {df_model.shape[0]} satır bulunmaktadır.")

df_model.isnull().sum()

In [ ]:
model_file = "formodel_dataset.csv"
df_model = pd.read_csv(model_file)


scaler = StandardScaler()
df_model["funding_total_usd_z"] = scaler.fit_transform(df_model[["funding_total_usd"]])


print("ilk 5 satır (Funding Total USD - Z Skoru uygulanmış):")
print(df_model[["funding_total_usd", "funding_total_usd_z"]].head())

df_model.to_csv(model_file, index=False)
print(f"z-Skoru uygulandı ve güncellenmiş veri {model_file} olarak kaydedildi.")

In [ ]:
model_file = "formodel_dataset.csv"
df_model = pd.read_csv(model_file)
status_counts = df_model["status"].value_counts()

# Status çeşitlerini ve frekanslarını yazdır
print("✅ Status Kategorileri ve Sayıları:")
print(status_counts)

In [ ]:
model_file = "formodel_dataset.csv"
df_model = pd.read_csv(model_file)
df_model = pd.get_dummies(df_model, columns=['status'], prefix='status')


print("One-Hot Encoding Sonrası İlk 5 Satır:")
print(df_model.head())

df_model.to_csv(model_file, index=False)

✅ One-Hot Encoding Sonrası İlk 5 Satır:
   funding_total_usd country_code           city  funding_rounds  \
0         10000000.0           in         Mumbai               1   
1           700000.0           us  Delaware City               2   
2          3406878.0      unknown        unknown               1   
3          2000000.0           cn        Beijing               1   
4         35000000.0           us      Champaign               1   

  first_funding_at last_funding_at  \
0       2015-01-05      2015-01-05   
1       2014-03-01      2014-10-14   
2       2014-01-30      2014-01-30   
3       2008-03-19      2008-03-19   
4       2014-07-24      2014-07-24   

                                  category_embedding  funding_total_usd_z  \
0  [ 1.27211273e-01  1.71060279e-01 -8.59424993e-...            -0.045030   
1  [-2.50366747e-01 -8.49269554e-02 -1.69406146e-...            -0.097742   
2  [ 8.08092859e-03 -2.04934537e-01 -8.56988057e-...            -0.082400   
3  [-8.1973988

In [ ]:
model_file = "formodel_dataset.csv"
df_model = pd.read_csv(model_file)


country_counts = df_model['country_code'].value_counts()


print("ülke Kodları ve Sayıları:")
print(country_counts.head(20))


print(f"toplam farklı ülke sayısı: {len(country_counts)}")

In [ ]:
model_file = "formodel_dataset.csv"
df_model = pd.read_csv(model_file)

# LabelEncoder oluşturma
country_encoder = LabelEncoder()
city_encoder = LabelEncoder()

# Country ve City için Label Encoding uygulama
df_model["country_code_encoded"] = country_encoder.fit_transform(df_model["country_code"])
df_model["city_encoded"] = city_encoder.fit_transform(df_model["city"])


print("ilk 10 Ülke Kodu (Encoded):")
print(df_model[["country_code", "country_code_encoded"]].head(10))

print("\n ilk 10 Şehir (Encoded):")
print(df_model[["city", "city_encoded"]].head(10))

df_model.to_csv(model_file, index=False)

In [ ]:
model_file = "formodel_dataset.csv"
df_model = pd.read_csv(model_file)

scaler = MinMaxScaler()
df_model["funding_rounds_scaled"] = scaler.fit_transform(df_model[["funding_rounds"]])


print("ilk 5 satır (Funding Rounds - Min-Max Scale uygulanmış):")
print(df_model[["funding_rounds", "funding_rounds_scaled"]].head())

df_model.to_csv(model_file, index=False)

✅ İlk 5 satır (Funding Rounds - Min-Max Scale uygulanmış):
   funding_rounds  funding_rounds_scaled
0               1               0.000000
1               2               0.055556
2               1               0.000000
3               1               0.000000
4               1               0.000000


In [ ]:
file_path = "formodel_dataset.csv"
df = pd.read_csv(file_path)


df['first_funding_at'] = pd.to_datetime(df['first_funding_at'], format='%Y-%m-%d', errors='coerce')
df['last_funding_at'] = pd.to_datetime(df['last_funding_at'], format='%Y-%m-%d', errors='coerce')


df['years_since_first_funding'] = (pd.Timestamp.today() - df['first_funding_at']).dt.days / 365
df['years_since_last_funding'] = (pd.Timestamp.today() - df['last_funding_at']).dt.days / 365


print(df[['first_funding_at', 'last_funding_at', 'years_since_first_funding', 'years_since_last_funding']].head())

df.to_csv(file_path, index=False)
print("✅ Dosya güncellendi! Tarih bazlı yıllar eklendi.")

In [ ]:
model_file = "formodel_dataset.csv"
df_model = pd.read_csv(model_file)


scaler = MinMaxScaler()
df_model[["years_since_first_funding_scaled", "years_since_last_funding_scaled"]] = scaler.fit_transform(
    df_model[["years_since_first_funding", "years_since_last_funding"]]
)


print("ilk 5 satır (Years Since First/Last Funding - Min-Max Scale uygulanmış):")
print(df_model[["years_since_first_funding", "years_since_first_funding_scaled",
                "years_since_last_funding", "years_since_last_funding_scaled"]].head())
df_model.to_csv(model_file, index=False)

In [ ]:

shutil.copy("formodel_dataset.csv", "/content/drive/My Drive/formodel_dataset.csv")
print("Dosya başarıyla Google Drive'a kaydedildi!")

In [ ]:
shutil.copy("preprocessed_startupdataset.csv", "/content/drive/My Drive/preprocessed_startupdataset.csv")
print("dosya başarıyla Google Drive'a kaydedildi!")

shutil.copy("updated_startupdataset.csv", "/content/drive/My Drive/updated_startupdataset.csv")
print("dosya başarıyla Google Drive'a kaydedildi!")


✅ Dosya başarıyla Google Drive'a kaydedildi!
✅ Dosya başarıyla Google Drive'a kaydedildi!


In [ ]:
model_file = "formodel_dataset.csv"
df_model = pd.read_csv(model_file)

# kaldırılacak sütunları tanımlama**
columns_to_drop = [
    "funding_total_usd",
    "country_code",
    "city",
    "funding_rounds",
    "first_funding_at",
    "last_funding_at",
    "years_since_first_funding",
    "years_since_last_funding"
]


df_model = df_model.drop(columns=columns_to_drop)


df_model.to_csv("formodel_csv.csv", index=False)



In [ ]:
df=pd.read_csv("formodel_csv.csv")
df.isnull().sum()

,0
category_embedding,0
funding_total_usd_z,0
status_acquired,0
status_closed,0
status_ipo,0
status_operating,0
country_code_encoded,0
city_encoded,0
funding_rounds_scaled,0
years_since_first_funding_scaled,3


In [ ]:
file_path = "formodel_csv.csv"
df = pd.read_csv(file_path)


status_columns = ["status_operating", "status_closed", "status_acquired", "status_ipo"]

# her sınıf için ağırlıklar
class_weights = {}
for col in status_columns:
    class_weights[col] = compute_class_weight(
        class_weight="balanced",
        classes=np.unique(df[col]),
        y=df[col]
    )

for col, weights in class_weights.items():
    print(f" {col} sınıf ağırlıkları: {dict(zip(np.unique(df[col]), weights))}")

In [ ]:
df = pd.read_csv("formodel_csv.csv")
print("ilk 5 veri:")
print(df.head())

In [ ]:



df = pd.read_csv("formodel_csv.csv")


def convert_embedding(x):
    try:
        return np.array(ast.literal_eval(x), dtype=np.float32)  # stringi float array'e çevir
    except:
        return np.nan

df["category_embedding"] = df["category_embedding"].astype(str).apply(convert_embedding)

# eksik category_embedding'leri Ortalama ile Doldur
valid_embeddings = df["category_embedding"].dropna().values
if len(valid_embeddings) > 0:
    valid_embeddings = np.stack(valid_embeddings)
    mean_embedding = np.mean(valid_embeddings, axis=0)  # ortalama embedding hesapla
else:
    mean_embedding = np.zeros(100)

df["category_embedding"] = df["category_embedding"].apply(lambda x: mean_embedding if isinstance(x, float) else x)

# 📌 category_embedding'i Ayrı Sütunlara Ayır (XGBoost İçin)
embedding_dim = len(mean_embedding)
embedding_cols = [f"embed_{i}" for i in range(embedding_dim)]

embedding_df = pd.DataFrame(df["category_embedding"].tolist(), columns=embedding_cols)
df = df.drop(columns=["category_embedding"])
df = pd.concat([df, embedding_df], axis=1)


X = df.drop(columns=["status_acquired", "status_closed", "status_ipo", "status_operating"])  # Hedef değişkenleri çıkar
y = df[["status_acquired", "status_closed", "status_ipo", "status_operating"]].values  # Çok sınıflı hedef değişken


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=np.argmax(y, axis=1))


unique_classes = np.array([0, 1, 2, 3])
y_train_labels = np.argmax(y_train, axis=1)
class_weights = compute_class_weight("balanced", classes=unique_classes, y=y_train_labels)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

model = XGBClassifier(n_estimators=100, objective="multi:softmax", num_class=4, random_state=42)
model.fit(X_train, y_train_labels, sample_weight=np.array([class_weight_dict[i] for i in y_train_labels]))



print("model başarıyla eğitildi, Test Sonuçları:", model.score(X_test, np.argmax(y_test, axis=1)))



✅ Model başarıyla eğitildi! Test Sonuçları: 0.5871150729335495


In [ ]:
y_pred = model.predict(X_test)


print("Classification Report:")
print(classification_report(np.argmax(y_test, axis=1), y_pred))


print(" Confusion Matrix:")
print(confusion_matrix(np.argmax(y_test, axis=1), y_pred))

📊 **Classification Report:**
              precision    recall  f1-score   support

           0       0.26      0.56      0.35       830
           1       0.23      0.51      0.32       905
           2       0.10      0.41      0.17       232
           3       0.93      0.60      0.73      7905

    accuracy                           0.59      9872
   macro avg       0.38      0.52      0.39      9872
weighted avg       0.79      0.59      0.65      9872

📊 **Confusion Matrix:**
[[ 461  162   85  122]
 [ 225  466   46  168]
 [  67   12   96   57]
 [1045 1385  702 4773]]


In [ ]:
import xgboost
print(xgboost.__version__)

2.1.3


In [ ]:
!pip install -U xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 MB 10.2 MB/s eta 0:00:00
  Attempting uninstall: xgboost
    Found existing installation: xgboost 2.1.3
    Uninstalling xgboost-2.1.3:
      Successfully uninstalled xgboost-2.1.3


In [ ]:
df = pd.read_csv("formodel_csv.csv")


def convert_embedding(x):
    try:
        return np.array(ast.literal_eval(x), dtype=np.float32)
    except:
        return np.nan

df["category_embedding"] = df["category_embedding"].astype(str).apply(convert_embedding)


valid_embeddings = df["category_embedding"].dropna().values
if len(valid_embeddings) > 0:
    valid_embeddings = np.stack(valid_embeddings)
    mean_embedding = np.mean(valid_embeddings, axis=0)
else:
    mean_embedding = np.zeros(100)

df["category_embedding"] = df["category_embedding"].apply(lambda x: mean_embedding if isinstance(x, float) else x)


embedding_dim = len(mean_embedding)
embedding_cols = [f"embed_{i}" for i in range(embedding_dim)]

embedding_df = pd.DataFrame(df["category_embedding"].tolist(), columns=embedding_cols)
df = df.drop(columns=["category_embedding"])
df = pd.concat([df, embedding_df], axis=1)


X = df.drop(columns=["status_acquired", "status_closed", "status_ipo", "status_operating"])
y = df[["status_acquired", "status_closed", "status_ipo", "status_operating"]].values


X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.30, random_state=42, stratify=np.argmax(y, axis=1))
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.50, random_state=42, stratify=np.argmax(y_temp, axis=1))

print(f"Eğitim Seti Boyutu: {len(X_train)}")
print(f"Doğrulama  Seti Boyutu: {len(X_val)}")
print(f"Test Seti Boyutu: {len(X_test)}")


unique_classes = np.array([0, 1, 2, 3])
y_train_labels = np.argmax(y_train, axis=1)
class_weights = compute_class_weight("balanced", classes=unique_classes, y=y_train_labels)


class_weights[0] *= 1.20
class_weights[1] *= 1.50
class_weights[2] *= 1.15
class_weights[3] *= 0.90

class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}


model = XGBClassifier(
    n_estimators=541,
    objective="multi:softmax",
    num_class=4,
    early_stopping_rounds = 10,
    random_state=42

)



model.fit(
    X_train,
    y_train_labels,
    eval_set=[(X_val, np.argmax(y_val, axis=1))],
    verbose=True,
    sample_weight=np.array([class_weight_dict[i] for i in y_train_labels])
)


results = model.evals_result()
print(results)

print(" Model başarıyla eğitildi, Test Sonuçları:", model.score(X_test, np.argmax(y_test, axis=1)))

y_pred = model.predict(X_test)


accuracy = accuracy_score(np.argmax(y_test, axis=1), y_pred)
print(f" Model Accuracy (Doğruluk): {accuracy:.4f}")


report = classification_report(np.argmax(y_test, axis=1), y_pred, digits=4)
print(" **Classification Report:**")
print(report)


conf_matrix = confusion_matrix(np.argmax(y_test, axis=1), y_pred)
print(" **Confusion Matrix:**")
print(conf_matrix)


macro_f1 = f1_score(np.argmax(y_test, axis=1), y_pred, average="macro")
print(f" Macro F1-Score: {macro_f1:.4f}")

y_prob = model.predict_proba(X_test)


roc_auc = roc_auc_score(y_test, y_prob, multi_class="ovr")
print(f" ROC-AUC Score: {roc_auc:.4f}")





Model başarıyla eğitildi Test Sonuçları: 0.6400931928687196
 Model Accuracy (Doğruluk): 0.6401
 **Classification Report:**
              precision    recall  f1-score   support

           0     0.2456    0.4717    0.3230       831
           1     0.2128    0.4316    0.2851       906
           2     0.1263    0.1558    0.1395       231
           3     0.8937    0.6959    0.7825      7904

    accuracy                         0.6401      9872
   macro avg     0.3696    0.4387    0.3825      9872
weighted avg     0.7587    0.6401    0.6831      9872

Confusion Matrix:
[[ 392  175   29  235]
 [ 203  391   16  296]
 [  50   22   36  123]
 [ 951 1249  204 5500]]
Macro F1-Score: 0.3825
ROC-AUC Score: 0.7665